In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data_path = 'data/match_4000710334517.csv'
data_df = pd.read_csv(data_path)
data_df.head()

,MatchID,SubID,RA,Dec,ImageID,SourceID,X,Y,Z
0,4000710334517,0,269.749758,-29.186608,22819,4001061039486,-0.003813,-0.873028,-0.487656
1,4000710334517,0,269.749758,-29.186608,44893,4001212780699,-0.003814,-0.873028,-0.487656
2,4000710334517,0,269.749758,-29.186608,44893,4001212780710,-0.003812,-0.873028,-0.487656
3,4000710334517,0,269.749758,-29.186608,44909,4001120525362,-0.003813,-0.873028,-0.487656
4,4000710334517,0,269.749758,-29.186608,44923,4001152086466,-0.003813,-0.873028,-0.487656


In [4]:
"""
    /// <summary>
    /// Returns the tangent vectors pointing to west and north.
    /// </summary>
    /// <param name="west">The west vector.</param>
    /// <param name="north">The north vector.</param>
    internal void Tangent(out vec3 west, out vec3 north)
    {
        double ra, dec, sinRa, cosRa, sinDec, cosDec;

        vec3.Cartesian2RadecRadian(this, out ra, out dec);
        sinRa = Math.Sin(ra);
        cosRa = Math.Cos(ra);
        sinDec = Math.Sin(dec);
        cosDec = Math.Cos(dec);

        west = new vec3(sinRa, -cosRa, 0, false);
        north = new vec3(-sinDec * cosRa, -sinDec * sinRa, cosDec, false);
    }
"""

def tangent(ra, dec):
    """Returns the tangent vectors pointing to west and north.

    Args:
        ra (float): right ascension in degrees
        dec (float): declination in degrees

    Returns:
        tuple[float,float]: west and north vectors
    """
    ra = np.radians(ra)
    dec = np.radians(dec)
    sinRa = np.sin(ra)
    cosRa = np.cos(ra)
    sinDec = np.sin(dec)
    cosDec = np.cos(dec)
    west = np.array([sinRa, -cosRa, 0])
    north = np.array([-sinDec * cosRa, -sinDec * sinRa, cosDec])
    return west, north